# Usage examples of the checkpointer

## Imports

In [ ]:
import time
from pathlib import Path

from checkpointer.checkpointer import Checkpointer



## Basic Counter Example

In this basic example we use a program, that simply counts to 10 000. 
Every 100 steps, a checkpoint is written to a file. 
Additionally , the checkpointer will react on the kernel interuption and create a checkpoint.

In [ ]:
# Initialize an checkpointer object
checkpointer = Checkpointer(
    local_checkpoint_file=Path("checkpoint.txt"),
    restore_function=lambda path: int(path.read_text()),
    checkpoint_function=lambda path, value: path.write_text(str(value)),
    checkpoint_every=100,
)


In [ ]:
start_value = checkpointer.restore(0)
print("starting at: ", start_value)
for i in range(start_value, 10_000):
    print(i)
    checkpointer.step(i)
    time.sleep(0.1)

print("finished counting")


## An example training using pytorch

 This example is based on Pytorch's basic example:
 https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#saving-models

In [ ]:
%pip install torch torchvision

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [ ]:

# Create data loaders.
batch_size = 256
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Get cpu, gpu or mps device for training.
device = ("cpu")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Setting up the checkpointer

In [ ]:
checkpointer = Checkpointer(
    local_checkpoint_file=Path("checkpoint.pt"),
    restore_function=lambda path: torch.load(path), # Load the model state dict
    checkpoint_function=lambda path, model: torch.save(model, path), # Save the model state dict
    checkpoint_every=1,
)

We can now run for some epochs, creating a checkpoint at the end of each:

In [ ]:
model = checkpointer.restore(model)
for t in range(2):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    checkpointer.checkpoint(model)


And at a later time, you can restore the model and continue training from where you left off.

In [ ]:
model = NeuralNetwork().to(device)
model = checkpointer.restore(model)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(2,10):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
    checkpointer.checkpoint(model)